## $Reduce\space Overfitting$

- $More data$

- $Constraint model complexity$

- $Dropout$

- $Data argumention$

- $Early Stopping$

###  $Regularization$

$L2-regularization$

```python
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(),lr = learning_rate,weight_decay = 0.01)
criteon = nn.CrossEntropyLoss().to(device)
```

$L1-regulatization$
```python
regulatization_loss = 0
for param in model.parameters():
    regulatization_loss += torch.sum(torch.abs(param))
classify_loss = criteon(logits,target)
loss = classify_loss + 0.01 * regulatization_loss

optimizer.zero_grad()
loss.backward()
optimizer.step()
```

In [3]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

from visdom import Visdom

batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('datasets/mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('datasets/mnist_data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)



class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.01)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Setting up a new session...


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.299677
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.272715
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.240755

Test set: Average loss: 0.0109, Accuracy: 4888.0/10000 (49%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.184309
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.089206
Train Epoch: 1 [40000/60000 (67%)]	Loss: 1.932384

Test set: Average loss: 0.0091, Accuracy: 5239.0/10000 (52%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.835482
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.713043
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.567164

Test set: Average loss: 0.0074, Accuracy: 5777.0/10000 (58%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.456117
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.354478
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.307719

Test set: Average loss: 0.0064, Accuracy: 6061.0/10000 (61%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.356326
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.256744
Train Epoch: 4 [40000/60000 (67%)]	Loss: 1.174796

Test set: Average loss